# CAI Data prep code

In [1]:
#importing necessary libraries
import os                   #to create new directories
import shutil               #to delete directories
import pandas as pd         #to convert excel data into dataframes into lists
from Bio import SeqIO       #to read/write the fasta files
import xlwt                 #to open and write into excel files .xls
from xlwt import Workbook   #to save excel files .xls
from openpyxl import load_workbook  #to edit pre existing excel files .xlsx
#from matplotlib import pyplot as plt    #to plot graphs
#import numpy as np                      #to deal with arrays

In [55]:
dataframe = pd.read_excel(r"C:\Users\husay\Desktop\Biotech\List of Genes (E.Coli K-12).xlsx")
gene_list=dataframe.values.tolist()

0       rfbC
1       yehE
2        tmk
3       yadG
4       sthA
        ... 
4759    fdoH
4760    yfiM
4761    guaB
4762    uxuA
4763    ariR
Name: Gene Name, Length: 4764, dtype: object
[1, 'yehE', 'b2112', 2192515.0, 2192796.0, 'DUF2574 domain-containing protein YehE', 'yehE', '93', '93 AA', 'Uncharacterized protein YehE', 'P33344']


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time

#load excel file
workbook = load_workbook(filename=r"C:\Users\husay\Desktop\Biotech\List of Genes (E.Coli K-12).xlsx")
#open workbook
sheet = workbook.active
num_loop=1
for gene in gene_list:
    #print(num_loop)
    if num_loop%10==0:
        workbook.save(filename=r"C:\Users\husay\Desktop\Biotech\List of Genes (E.Coli K-12).xlsx")
    
    gene_name=gene[1]
    
    #--| Setup
    options = Options()
    #options.add_argument("--headless")
    #options.add_argument("--window-size=1980,1020")
    service=Service(executable_path=r'C:\Windows\chromedriver.exe')
    browser = webdriver.Chrome(service=service, options=options)
    #--| Parse or automation
    url = "https://www.uniprot.org/uniprotkb?query="+str(gene_name)
    browser.get(url)
    time.sleep(2)
 
    # Use Bs to Parse
    soup = BeautifulSoup(browser.page_source, 'html5lib')
    
    #use find and find_all to get to the table
    first = soup.find('table', attrs={'class':'hotjar-margin Anr5j data-table'})
    if first==None: continue
    if not first==None:
        second= first.find_all('tr')
    #print(len(second))
    #browser.close()
    if second==None: continue

    strain_check=False
    # here [j] is the row no.
    #The web scraper gets the first 25 rows of the table
    for j in range(len(second)):
        #checking if gene name is correct or not
        if second[j]==None:break
        c=second[j].find_all('span', attrs={'class':'FR3zN'})
        cc=str(c)      #convert bs.element.tag to string
        gene_check = (gene_name==cc[21:-8])
    
        #using find_all method to get all 'td' ie. table elements
        #s and c and dd are variables used
        c=second[j].find_all('td')
        info_gene=[] #to store info ['entry','entry_name','protein_name','gene_names','organism','length','type of protein']
        for s in c:
            #converting bs4.element.tag to string
            dd=str(s)
            #ignoring empty strings
            if not dd=='[]':
                #finding indices of < and > to get info in middle
                indices1 = [i for i in range(len(dd)) if dd[i] == '<']
                indices2 = [i for i in range(len(dd)) if dd[i] == '>']
                for i in range(len(indices1)-1):
                    if indices1[i+1]-indices2[i]!=1:
                        info_gene.append(dd[indices2[i]+1:indices1[i+1]])
                        
        #Checking that organism is E.coli
            
        if 'Escherichia coli (strain K12)' in info_gene and gene_check==True:
            strain_check=True
            break; #exit loop because we are on the correct row
    #print(info_gene)
    #modify the desired cell
    if strain_check==False:
        continue
    sheet["H"+str(gene_list.index(gene)+2)] = (info_gene[-1][:-3])   #no. of aa
    #protein_name=str(info_gene[2])
    sheet["J"+str(gene_list.index(gene)+2)] = info_gene[2]   #name of protein
    sheet["K"+str(gene_list.index(gene)+2)] = info_gene[0]    #entry tag
    
    #--| Setup
    options = Options()
    #options.add_argument("--headless")
    #options.add_argument("--window-size=1980,1020")
    service=Service(executable_path=r'C:\Windows\chromedriver.exe')
    browser = webdriver.Chrome(service=service, options=options)
    #--| Parse or automation
    url = "https://www.uniprot.org/uniprotkb/"+str(info_gene[0])+"/entry#subcellular_location"
    browser.get(url)
    time.sleep(2)
 
    # Use Bs to Parse
    soup = BeautifulSoup(browser.page_source, 'html5lib')
    #attrs={'href':'/locations/SL-0037'}
    first = soup.find_all(attrs={'taxid':'83333'})
    #finding indices of < and > to get info in middle
    dd=str(first)
    indices1 = [i for i in range(len(dd)) if dd[i] == '<']
    indices2 = [i for i in range(len(dd)) if dd[i] == '>']
    type_found=0
    for i in range(len(indices1)-1):
        if type_found==1: break
        if indices1[i+1]-indices2[i]!=1:
            type_found=1
            info_gene.append(dd[indices2[i]+1:indices1[i+1]])
            print(dd[indices2[i]+1:indices1[i+1]],'\n')
    
    sheet["I"+str(gene_list.index(gene)+2)] = (info_gene[-1])   #type of protein
    #print(info_gene,'\n')
    num_loop+=1
    

In [15]:
#save the file
workbook.save(filename=r"C:\Users\husay\Desktop\Biotech\List of Genes (E.Coli K-12).xlsx")

In [2]:
dataframe = pd.read_excel(r"C:\Users\husay\Desktop\Biotech\New_final_list.xlsx")
print(dataframe)
gene_list=dataframe.values.tolist()

      Index Gene Name Accession-1  Left End Position  Right End Position  \
0         1      rfbC       b2038          2109581.0           2110138.0   
1         2       tmk       b1098          1155124.0           1155765.0   
2         3      yadG       b0127           142779.0            143705.0   
3         4      sthA       b3962          4159390.0           4160790.0   
4         5       ndh       b1109          1166085.0           1167389.0   
...     ...       ...         ...                ...                 ...   
2559   2560      yihV       b3883          4073739.0           4074635.0   
2560   2561      ycjV       b1318          1380821.0           1381789.0   
2561   2562      galF       b2042          2113434.0           2114327.0   
2562   2563      fdoH       b3893          4081857.0           4082759.0   
2563   2564      guaB       b2508          2632604.0           2634070.0   

                                                Product Map to E.coli Genome  \
0      

In [3]:
print(gene_list[0])

[1, 'rfbC', 'b2038', 2109581.0, 2110138.0, 'dTDP-4-dehydrorhamnose 35-epimerase', nan, 185, 'cytosol', 'dTDP-4-dehydrorhamnose 35-epimerase', 'P37745', 1529]


In [4]:
protein_type = {''}
for i in range(len(gene_list)):
    protein_type.add(gene_list[i][8])
    #print(gene_list[i][8])

In [5]:
print((protein_type))
pt=protein_type.copy()
print(type(pt))

{'', 'outer membrane', 'host outer membrane', 'Cell membrane', 'Membrane', 'pilus', 'cytoplasm ', 'Cytoplasm nucleoid', 'Bacterial flagellum basal body', 'cell pole', 'Bacterial microcompartment', 'cytoplasmic replication fork', 'Cytoplasm', 'bacterial nucleoid', 'cytosol', 'Cytoplasm cytoskeleton', 'Cell inner membrane', 'plasma membrane', 'outer membrane-bounded periplasmic space', 'cytosolic ribosome', 'Periplasm', 'cytoplasmic side of plasma membrane', 'bacterial-type flagellum hook', 'cell outer membrane', 'Cell outer membrane', 'membrane', 'chromosome', 'cytoplasm', 'Cytoplasm cytosol', 'Fimbrium', 'bacterial-type flagellum', 'Secreted'}
<class 'set'>


In [6]:
for i in pt:
    if str(type(i))=="<class 'str'>":
        if 'AA' in i:
            protein_type.remove(i)

In [84]:
location_of_protein =['Membrane', 'Cell outer membrane','pilus', 'cytoplasmic replication fork','Bacterial flagellum basal body',
'host outer membrane','bacterial nucleoid', 'plasma membrane', 'Cytoplasm, nucleoid','Periplasm','Cytoplasm',
'Fimbrium','cell outer membrane', 'replisome','outer membrane','Cytoplasm, cytoskeleton','Bacterial microcompartment',
'Cell membrane', 'Cytoplasm, cytosol','cytosol','outer membrane-bounded periplasmic space','membrane',
'bacterial-type flagellum hook',
'cytoplasmic side of plasma membrane', 'cytosolic ribosome','cytoplasm','Cell inner membrane','bacterial-type flagellum']

In [85]:
print(location_of_protein)

['Membrane', 'Cell outer membrane', 'pilus', 'cytoplasmic replication fork', 'Bacterial flagellum basal body', 'host outer membrane', 'bacterial nucleoid', 'plasma membrane', 'Cytoplasm, nucleoid', 'Periplasm', 'Cytoplasm', 'Fimbrium', 'cell outer membrane', 'replisome', 'outer membrane', 'Cytoplasm, cytoskeleton', 'Bacterial microcompartment', 'Cell membrane', 'Cytoplasm, cytosol', 'cytosol', 'outer membrane-bounded periplasmic space', 'membrane', 'cytoplasmic side of plasma membrane', 'cytosolic ribosome', 'cytoplasm', 'Cell inner membrane']


In [78]:
drop_index=[] #these are the indices which needed to be removed from the excel file
for i in range(len(gene_list)):
    if not gene_list[i][8] in plist:
        drop_index.append(i)

In [80]:
df1=dataframe.drop(dataframe.index[drop_index])
#print(df1)

      Unnamed: 0 Gene Name Accession-1  Left End Position  Right End Position  \
0              0      rfbC       b2038          2109581.0           2110138.0   
2              2       tmk       b1098          1155124.0           1155765.0   
3              3      yadG       b0127           142779.0            143705.0   
4              4      sthA       b3962          4159390.0           4160790.0   
5              5       ndh       b1109          1166085.0           1167389.0   
...          ...       ...         ...                ...                 ...   
4752        4752      yihV       b3883          4073739.0           4074635.0   
4757        4757      ycjV       b1318          1380821.0           1381789.0   
4758        4758      galF       b2042          2113434.0           2114327.0   
4759        4759      fdoH       b3893          4081857.0           4082759.0   
4761        4761      guaB       b2508          2632604.0           2634070.0   

                           

In [81]:
# writing to Excel
datatoexcel = pd.ExcelWriter(r'C:\Users\husay\Desktop\Biotech\Final_list.xlsx')
  
# write DataFrame to excel
df1.to_excel(datatoexcel)
  
# save the excel
datatoexcel.save()

C:\Users\husay\AppData\Local\Temp\ipykernel_8440\3678194010.py:8: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  datatoexcel.save()


In [92]:
dataframe = pd.read_excel(r"C:\Users\husay\Desktop\Biotech\New_final_list.xlsx")
print(dataframe)
gene_list=dataframe.values.tolist()

      Index Gene Name Accession-1  Left End Position  Right End Position  \
0         1      rfbC       b2038          2109581.0           2110138.0   
1         2       tmk       b1098          1155124.0           1155765.0   
2         3      yadG       b0127           142779.0            143705.0   
3         4      sthA       b3962          4159390.0           4160790.0   
4         5       ndh       b1109          1166085.0           1167389.0   
...     ...       ...         ...                ...                 ...   
2559   2560      yihV       b3883          4073739.0           4074635.0   
2560   2561      ycjV       b1318          1380821.0           1381789.0   
2561   2562      galF       b2042          2113434.0           2114327.0   
2562   2563      fdoH       b3893          4081857.0           4082759.0   
2563   2564      guaB       b2508          2632604.0           2634070.0   

                                                Product Map to E.coli Genome  \
0      

In [102]:
print(gene_list[2][9])

Uncharacterized ABC transporter ATP-binding protein YadG
